In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import re
import glob, os, shutil
import gzip
import random
from bs4 import BeautifulSoup
import json
import pandas as pd
import subprocess

In [3]:
from src import feature_extraction

In [4]:
from src import matrix_construction

In [5]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

import numpy as np

In [6]:
with open('config/data-params.json') as f:
    params = json.load(f)
sitemap = params['sitemap']
out = params['path']
num = params['num']
cat = params['cat']
malware_path = params['malware']

In [7]:
def get_malware_paths(malware_path, num):
    paths = []
    count = 0
    for d, dirs, files in os.walk(malware_path):
        for subd in dirs:
            if subd == 'smali' and count < num:
                paths.append(d)
                count += 1
            if count > num:
                break
    return paths

In [8]:
mal_paths = feature_extraction.get_malware_paths(malware_path, 40)

In [9]:
mal_smali = feature_extraction.all_smali(mal_paths)

In [10]:
mal_smali_df = pd.DataFrame(mal_smali)
mal_smali_df['malware'] = 1

In [11]:
benign_smali1 = feature_extraction.all_smali(feature_extraction.get_sub_dir(out + '/' + cat[0]))

In [12]:
benign_smali2 = feature_extraction.all_smali(feature_extraction.get_sub_dir(out + '/' + cat[1]))

In [13]:
benign1 = pd.DataFrame(benign_smali1)
benign2 = pd.DataFrame(benign_smali2)

In [14]:
all_benign = benign1.append(benign2, ignore_index = True)
all_benign['malware'] = 0

In [15]:
entire = all_benign.append(mal_smali_df, ignore_index = True)

In [16]:
entire = entire.sample(frac=1, axis=0).reset_index(drop=True)

In [17]:
y_train = entire['malware'][:60]
y_test = entire['malware'][60:]
entire = entire.drop(['malware'], axis = 1)

In [18]:
X_train = entire[:60]
X_test = entire[60:]

In [19]:
A, B, P, A_test = matrix_construction.A_B_P(X_train, X_test)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [18]:
A, apis = matrix_construction.A(X_train)

In [19]:
B = matrix_construction.B(entire)

In [20]:
P = matrix_construction.P(apis)

In [27]:
P

<379141x379141 sparse matrix of type '<class 'numpy.int64'>'
	with 7629147 stored elements in Compressed Sparse Row format>

In [28]:
B

<379141x379141 sparse matrix of type '<class 'numpy.int64'>'
	with 10014943 stored elements in Compressed Sparse Row format>

In [29]:
A

<77x379141 sparse matrix of type '<class 'numpy.int64'>'
	with 860785 stored elements in Compressed Sparse Row format>

In [30]:
svc = SVC(kernel='precomputed')


In [34]:
kernel_train = A.dot(B).dot(P).dot(B).dot(A.T).todense()

In [ ]:
svc.fit(kernel_train, y_train)

In [ ]:
y_pred = svc.predict(kernel_train)
accuracy_score(y_train, y_pred)

In [ ]:
def kernel_train(A, B, P):
    return [A.dot(A.T), A.dot(B).dot(A.T), A.dot(P).dot(A.T), A.dot(B).dot(P).dot(B).dot(A.T)]

In [ ]:
def kernel_test(A_train, A_test, B_test, P_test):
    return [A_test.dot(A_train.T), A_test.dot(B_test).dot(A_train.T), A_test.dot(P_test).dot(A_train.T), A_test.dot(B_test).dot(P_test).dot(B_test).dot(A_train.T)]

In [ ]:
def compute_metrics(mat):
    return mat + [(mat[0]+mat[3])/sum(mat), mat[2]/(mat[2]+mat[3])]